In [1]:
# !pip install transformers==4.30.2

In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf 
from sklearn.model_selection import train_test_split
from transformers import  MT5TokenizerFast, TFAutoModelForSeq2SeqLM
from tensorflow.keras.callbacks import EarlyStopping
import re
import warnings
warnings.filterwarnings("ignore")

In [3]:
def get_data(url: str, n_rows: int):
    data = pd.read_csv(url, names=["target", "input"], nrows=n_rows)
    data = data.sample(frac=1).copy()
    data = data.loc[(~data["input"].isna()) & (~data["target"].isna())].copy()
    data = data[['input', 'target']].reset_index(drop=True).copy()
    return data

In [4]:
# https://www.kaggle.com/datasets/disisbig/hindi-text-short-summarization-corpus
data = get_data(url="train.csv", n_rows=80000)

In [5]:
data.head()

,input,target
0,22 साल की दीपा कर्माकर ओलंपिक के क्वालीफाई करन...,"जानिए, ओलंपिक के लिए क्वालीफाई कर इतिहास रचने ..."
1,राफेल मामले पर सुप्रीम कोर्ट से केंद्र सरकार क...,"राफेल डील पर मोदी सरकार को झटका, दोबारा सुनवाई..."
2,बीमार राहुल राज की पटना से दिल्ली तक की\nजीवनर...,जीवनरक्षक यात्रा घातक साबित हुई
3,यहां जानें दिनभर क्या रहा टेक जगत का हाल. हम य...,Tech Wrap: यहां जानें आज की 5 बड़ी खबरें
4,नरेंद्र मोदी पर तीखे प्रहार करने के बाद केंद्र...,मैं व्यक्तिगत तौर पर मोदी के खिलाफ नहीं: फारूक...


In [6]:
def split_data(data: pd.DataFrame, input_col: str="input", target_col: str="target", test_size: float=0.1):
    x_train, x_test, y_train, y_test = train_test_split(data[input_col], data[target_col], 
                                                        random_state=42, test_size=test_size)
    
    print(f'x_train.shape: {x_train.shape}, x_test.shape: {x_test.shape}, '+
          f'y_train.shape: {y_train.shape}, y_test.shape: {y_test.shape}')
    x_train, x_test, y_train, y_test = x_train.to_list(), x_test.to_list(), y_train.to_list(), y_test.to_list()
    return x_train, x_test, y_train, y_test

In [7]:
x_train, x_test, y_train, y_test = split_data(data=data)

x_train.shape: (71993,), x_test.shape: (8000,), y_train.shape: (71993,), y_test.shape: (8000,)


In [8]:
x_train[0], y_train[0]

('दिवाली के बाद से गैस चैम्बर बन गई है दिल्ली\nवायु प्रदूषण कम होने से देशवासियों की बढ़ेगी उम्र\nआगरा में 8.1 साल और बरेली में 7.8 साल ज्यादा होगी उम्र\nदिवाली की आतिशबाजी के चलते राष्ट्रीय राजधानी दिल्ली समेत पूरे देश की आबोहवा खराब हो गई है. दिल्ली-एनसीआर गैस चैम्बर बन गया है और लोगों का सांस लेना दूभर हो गया है. इतना ही नहीं, वायु प्रदूषण के चलते लोगों की जिंदगी पर संकट बढ़ता जा रहा है.\nशिकागो यूनिवर्सिटी के एनर्जी पॉलिसी इंस्टीट्यूट द्वारा जारी एयर क्वालिटी लाइफ इंडेक्स (AQLI) के मुताबिक अगर भारत में वायु प्रदूषण कम रहता है और वायु गुणवत्ता विश्व स्वास्थ्य संगठन (WHO) के मानकों अनुसार रहती है, तो लोग औसतन चार साल ज्यादा जीवित रह सकते हैं.\nएयर क्वालिटी लाइफ इंडेक्स में कहा गया कि\nदिल्ली-एनसीआर\nसमेत भारत के सबसे अधिक आबादी वाले शहरों में वायु प्रदूषण की वजह से लोगों की जिंदगी 9 साल तक कम होती जा रही है. अगर इन शहरों में प्रदूषण कम हो जाए, तो यहां रहने वाले लोगों की जिंदगी 9 साल तक बढ़ जाएगी. वहीं, अगर आगरा में वायु प्रदूषण कम हो जाए, तो वहां के लोगों की जिंदगी 8.1 वर्ष और बरेली 

In [9]:
strategy = tf.distribute.MirroredStrategy()
CHECKPOINT = "google/mt5-small"
INPUT_N_TOKENS = 300 # considering only 300 tokens due to memory constraints
TARGET_N_TOKENS = 100 # considering only 100 since it's only headline
# ideal way of computing N_TOKENS is to analyze the lengths of tokens of few samples in input & target
BATCH_SIZE = 16 * strategy.num_replicas_in_sync

In [10]:
def tokenize(input: list, target: list, input_n_tokens: int, target_n_tokens: int):
    tokenizer = MT5TokenizerFast.from_pretrained(CHECKPOINT)
#     print(f'Example:\n{input[0]}\n{tokenizer.tokenize(input[0])}')
    tokenized_data = tokenizer(text=input, max_length=input_n_tokens, truncation=True, padding="max_length")
    tokenized_data["labels"] = tokenizer(text_target=target, max_length=target_n_tokens, truncation=True, padding="max_length")["input_ids"]
    return tokenized_data    

In [11]:
tokenize(input=x_train[:2], target=y_train[:2], input_n_tokens=INPUT_N_TOKENS, 
         target_n_tokens=TARGET_N_TOKENS)

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. If you see this, DO NOT PANIC! This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thouroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


{'input_ids': [[36153, 49304, 641, 259, 7820, 1162, 1462, 50306, 6158, 4439, 63707, 10022, 1462, 1665, 830, 29532, 4315, 66497, 2047, 51447, 46589, 8583, 1248, 757, 1162, 11287, 36162, 6711, 975, 1822, 15519, 60858, 2139, 92886, 943, 157962, 844, 37732, 259, 17494, 259, 1367, 139353, 90559, 844, 130351, 259, 17494, 259, 25758, 4096, 1248, 9988, 2139, 92886, 36153, 49304, 975, 90377, 59788, 65809, 1304, 641, 14608, 1484, 259, 41899, 11392, 24128, 1304, 29532, 259, 32423, 10682, 4324, 11287, 975, 943, 74078, 3286, 5362, 4648, 59582, 1248, 1462, 1665, 830, 260, 29532, 264, 45475, 3235, 37966, 1462, 50306, 6158, 4439, 63707, 10022, 1462, 2075, 830, 259, 1367, 5724, 1762, 1396, 1163, 4736, 25488, 7556, 1437, 14391, 34214, 1248, 1462, 2075, 830, 260, 37518, 1437, 259, 2017, 3535, 261, 4315, 66497, 2047, 51447, 46589, 641, 14608, 1484, 5724, 1762, 1396, 975, 8134, 19033, 9988, 1587, 3028, 38102, 1822, 15519, 1114, 2297, 259, 11255, 830, 260, 36689, 1770, 37290, 22918, 116884, 21954, 52921, 64

In [12]:
tokenized_train = tokenize(input=x_train, target=y_train, input_n_tokens=INPUT_N_TOKENS, 
         target_n_tokens=TARGET_N_TOKENS)
tokenized_test = tokenize(input=x_test, target=y_test, input_n_tokens=INPUT_N_TOKENS, 
         target_n_tokens=TARGET_N_TOKENS)

In [13]:
tokenized_train[0]

Encoding(num_tokens=300, attributes=[ids, type_ids, tokens, offsets, attention_mask, special_tokens_mask, overflowing])

In [14]:
tokenized_train[:5]

[Encoding(num_tokens=300, attributes=[ids, type_ids, tokens, offsets, attention_mask, special_tokens_mask, overflowing]),
 Encoding(num_tokens=300, attributes=[ids, type_ids, tokens, offsets, attention_mask, special_tokens_mask, overflowing]),
 Encoding(num_tokens=300, attributes=[ids, type_ids, tokens, offsets, attention_mask, special_tokens_mask, overflowing]),
 Encoding(num_tokens=300, attributes=[ids, type_ids, tokens, offsets, attention_mask, special_tokens_mask, overflowing]),
 Encoding(num_tokens=300, attributes=[ids, type_ids, tokens, offsets, attention_mask, special_tokens_mask, overflowing])]

In [15]:
def return_tf_tensors(data):
    data = tf.data.Dataset.from_tensor_slices(dict(data))
    data = data.prefetch(tf.data.AUTOTUNE)
    return data

In [16]:
train_tf_data = return_tf_tensors(tokenized_train)
test_tf_data = return_tf_tensors(tokenized_test)

In [17]:
for i in train_tf_data.take(1):
    print(i)

{'input_ids': <tf.Tensor: shape=(300,), dtype=int32, numpy=
array([ 36153,  49304,    641,    259,   7820,   1162,   1462,  50306,
         6158,   4439,  63707,  10022,   1462,   1665,    830,  29532,
         4315,  66497,   2047,  51447,  46589,   8583,   1248,    757,
         1162,  11287,  36162,   6711,    975,   1822,  15519,  60858,
         2139,  92886,    943, 157962,    844,  37732,    259,  17494,
          259,   1367, 139353,  90559,    844, 130351,    259,  17494,
          259,  25758,   4096,   1248,   9988,   2139,  92886,  36153,
        49304,    975,  90377,  59788,  65809,   1304,    641,  14608,
         1484,    259,  41899,  11392,  24128,   1304,  29532,    259,
        32423,  10682,   4324,  11287,    975,    943,  74078,   3286,
         5362,   4648,  59582,   1248,   1462,   1665,    830,    260,
        29532,    264,  45475,   3235,  37966,   1462,  50306,   6158,
         4439,  63707,  10022,   1462,   2075,    830,    259,   1367,
         5724,   

In [18]:
def fit_model(train_data, val_data, epochs=2, eta=1e-4, early_stopping_patience=1, batch_size=BATCH_SIZE):
    with strategy.scope():
        model = TFAutoModelForSeq2SeqLM.from_pretrained(CHECKPOINT)
        model.compile(optimizer=tf.keras.optimizers.Adam(eta))

    print(model.summary())
    early_stop = EarlyStopping(monitor="val_loss", patience=early_stopping_patience, mode="min")
    model.fit(train_data.shuffle(len(train_data)).batch(batch_size), validation_data=val_data.shuffle(len(val_data)).batch(batch_size), 
          epochs=epochs, callbacks=[early_stop])
    return model

In [19]:
model = fit_model(train_data=train_tf_data, val_data=test_tf_data)

All model checkpoint layers were used when initializing TFMT5ForConditionalGeneration.

All the layers of TFMT5ForConditionalGeneration were initialized from the model checkpoint at google/mt5-small.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFMT5ForConditionalGeneration for predictions without further training.


Model: "tfmt5_for_conditional_generation"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 shared (Embedding)          multiple                  128057344 
                                                                 
 encoder (TFT5MainLayer)     multiple                  146940608 
                                                                 
 decoder (TFT5MainLayer)     multiple                  153236160 
                                                                 
 lm_head (Dense)             multiple                  128057344 
                                                                 
Total params: 300,176,768
Trainable params: 300,176,768
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/2
2250/2250 [==============================] - 2937s 1s/step - loss: 3.5900 - val_loss: 1.0923
Epoch 2/2
2250/2250 [=======================

**INFERENCE**

In [20]:
def inference_tokenize(input: list, n_tokens: int):
    tokenizer = MT5TokenizerFast.from_pretrained(CHECKPOINT)
    tokenized_data = tokenizer(text=input, max_length=n_tokens, truncation=True, padding="max_length", return_tensors="tf")
    return tokenizer, tokenized_data    

In [23]:
def inference(txt: str):
    test_data = [txt]
    inference_tokenizer, tokenized_data = inference_tokenize(input=test_data, n_tokens=INPUT_N_TOKENS)
    pred = model.generate(**tokenized_data, max_new_tokens=TARGET_N_TOKENS)
    result = inference_tokenizer.decode(pred[0])
    result = re.sub("<.*?>", "", result).strip()
    print(f"INPUT TEXT:\n{txt}\n\nHEADLINE:\n{result}")
    return (txt, result)

In [24]:
txt = '''
जम्मू के अरनिया सेक्टर में इंटरनेशनल बॉर्डर पर 26 अक्टूबर की शाम 8 बजे पाकिस्तानी रेंजर्स ने बेवजह फायरिंग शुरू कर दी। BSF भी इसके जवाब में फायरिंग कर रही है। न्यूज एजेंसी PTI के मुताबिक, गोलीबारी में BSF के कुछ जवान घायल हुए हैं, हालांकि इसकी कोई आधिकारिक पुष्टि नहीं हुई है। जानकारी मिलते ही BSF के सीनियर अधिकारी मौके पर पहुंच रहे हैं।

BSF के एक सीनियर अधिकारी ने बताया कि 17 अक्टूबर को भी पाकिस्तानी रेंजर्स ने अरनिया सेक्टर में रात करीब 8 बजे बिना वजह के गोलीबारी की थी। जिसमें BSF के दो जवान घायल हो गए थे। 25 फरवरी 2021 के बाद से पाकिस्तान जम्मू-कश्मीर के अलावा कई अन्य क्षेत्रों में सीजफायर का उल्लंघन कर चुका है।
'''
txt, result = inference(txt)

INPUT TEXT:

जम्मू के अरनिया सेक्टर में इंटरनेशनल बॉर्डर पर 26 अक्टूबर की शाम 8 बजे पाकिस्तानी रेंजर्स ने बेवजह फायरिंग शुरू कर दी। BSF भी इसके जवाब में फायरिंग कर रही है। न्यूज एजेंसी PTI के मुताबिक, गोलीबारी में BSF के कुछ जवान घायल हुए हैं, हालांकि इसकी कोई आधिकारिक पुष्टि नहीं हुई है। जानकारी मिलते ही BSF के सीनियर अधिकारी मौके पर पहुंच रहे हैं।

BSF के एक सीनियर अधिकारी ने बताया कि 17 अक्टूबर को भी पाकिस्तानी रेंजर्स ने अरनिया सेक्टर में रात करीब 8 बजे बिना वजह के गोलीबारी की थी। जिसमें BSF के दो जवान घायल हो गए थे। 25 फरवरी 2021 के बाद से पाकिस्तान जम्मू-कश्मीर के अलावा कई अन्य क्षेत्रों में सीजफायर का उल्लंघन कर चुका है।


HEADLINE:
पाकिस्तानी रेंजर्स ने बेवजह फायरिंग


In [25]:
txt = '''
कतर की एक अदालत ने भारत के 8 पूर्व नौसैनिकों को मौत की सजा सुनाई है। ये एक साल से कतर की अलग-अलग जेलों में कैद हैं। न्यूज एजेंसी PTI की रिपोर्ट के मुताबिक भारत सरकार ने इस पर हैरानी जाहिर की है। उन्हें छुड़ाने के लिए कानूनी रास्ते खोजे जा रहे हैं। विदेश मंत्रालय ने बयान जारी कर कहा है कि हम जजमेंट की डिटेलिंग का इंतजार कर रहे हैं।

कतर सरकार ने 8 भारतीयों पर लगे आरोपों को सार्वजनिक नहीं किया है। कतर में जिन 8 पूर्व नौसेना अफसरों को मौत की सजा दी गई है उनके नाम हैं- कैप्टन नवतेज सिंह गिल, कैप्टन सौरभ वशिष्ठ, कैप्टन बीरेंद्र कुमार वर्मा, कमांडर पूर्णेन्दु तिवारी, कमांडर सुग्नाकर पकाला, कमांडर संजीव गुप्ता, कमांडर अमित नागपाल और सेलर रागेश।
'''
txt, result = inference(txt)

INPUT TEXT:

कतर की एक अदालत ने भारत के 8 पूर्व नौसैनिकों को मौत की सजा सुनाई है। ये एक साल से कतर की अलग-अलग जेलों में कैद हैं। न्यूज एजेंसी PTI की रिपोर्ट के मुताबिक भारत सरकार ने इस पर हैरानी जाहिर की है। उन्हें छुड़ाने के लिए कानूनी रास्ते खोजे जा रहे हैं। विदेश मंत्रालय ने बयान जारी कर कहा है कि हम जजमेंट की डिटेलिंग का इंतजार कर रहे हैं।

कतर सरकार ने 8 भारतीयों पर लगे आरोपों को सार्वजनिक नहीं किया है। कतर में जिन 8 पूर्व नौसेना अफसरों को मौत की सजा दी गई है उनके नाम हैं- कैप्टन नवतेज सिंह गिल, कैप्टन सौरभ वशिष्ठ, कैप्टन बीरेंद्र कुमार वर्मा, कमांडर पूर्णेन्दु तिवारी, कमांडर सुग्नाकर पकाला, कमांडर संजीव गुप्ता, कमांडर अमित नागपाल और सेलर रागेश।


HEADLINE:
भारतीयों को मौत की सजा सुनाई


In [26]:
txt = '''
नेशनल गेम्स के 37वें सीजन की शुरुआत 25 अक्टूबर से हो चुकी है। नेशनल गेम्स की ऑफिशियल ओपनिंग सेरेमनी आज (26 अक्टूबर) गोवा के जवाहरलाल नेहरू स्टेडियम में शुरू हुई। पीएम नरेंद्र मोदी शाम करीब 6:45 बजे स्टेडियम पहुंचे। उन्होंने कहा, देश 2030 में यूथ ओलिंपिक्स और 2036 में ओलिंपिक्स की मेजबानी के लिए तैयार है।

पीएम मोदी को नेशनल हॉकी टीम के कप्तान हरमनप्रीत सिंह ने नेशनल गेम्स की मशाल सौंपी। पीएम ने मशाल को स्टेज पर स्थापित करने के बाद दर्शकों को संबोधित किया।

ओलिंपिक्स की मेजबानी के लिए तैयार भारत
मोदी ने कहा , 'भारत साल 2030 में यूथ ओलिंपिक्स और 2036 में ओलिंपिक खेलों की मेजबानी के लिए पूरी तरह से तैयार है। गोवा में जो स्पोर्ट्स का इंफ्रास्ट्रक्चर तैयार हुआ है, इससे नए खिलाड़ियों को तैयारी करने का मौका मिलेगा।'
'''
txt, result = inference(txt)

INPUT TEXT:

नेशनल गेम्स के 37वें सीजन की शुरुआत 25 अक्टूबर से हो चुकी है। नेशनल गेम्स की ऑफिशियल ओपनिंग सेरेमनी आज (26 अक्टूबर) गोवा के जवाहरलाल नेहरू स्टेडियम में शुरू हुई। पीएम नरेंद्र मोदी शाम करीब 6:45 बजे स्टेडियम पहुंचे। उन्होंने कहा, देश 2030 में यूथ ओलिंपिक्स और 2036 में ओलिंपिक्स की मेजबानी के लिए तैयार है।

पीएम मोदी को नेशनल हॉकी टीम के कप्तान हरमनप्रीत सिंह ने नेशनल गेम्स की मशाल सौंपी। पीएम ने मशाल को स्टेज पर स्थापित करने के बाद दर्शकों को संबोधित किया।

ओलिंपिक्स की मेजबानी के लिए तैयार भारत
मोदी ने कहा , 'भारत साल 2030 में यूथ ओलिंपिक्स और 2036 में ओलिंपिक खेलों की मेजबानी के लिए पूरी तरह से तैयार है। गोवा में जो स्पोर्ट्स का इंफ्रास्ट्रक्चर तैयार हुआ है, इससे नए खिलाड़ियों को तैयारी करने का मौका मिलेगा।'


HEADLINE:
नेशनल गेम्स की मशाल सौंपी


**SAVE MODEL**

In [27]:
model.save_weights("mt5_hindi_summarizer.h5", save_format="h5")

**LOAD SAVED MODEL**

In [28]:
CHECKPOINT = "google/mt5-small"
INPUT_N_TOKENS = 300
TARGET_N_TOKENS = 100

In [29]:
def prod_inference_tokenize(input: list, n_tokens: int):
    tokenizer = MT5TokenizerFast.from_pretrained(CHECKPOINT)
    tokenized_data = tokenizer(text=input, max_length=n_tokens, truncation=True, padding="max_length", return_tensors="tf")
    return tokenizer, tokenized_data    

In [30]:
loaded_model = TFAutoModelForSeq2SeqLM.from_pretrained(CHECKPOINT)
loaded_model.load_weights("mt5_hindi_summarizer.h5", by_name=True)

All model checkpoint layers were used when initializing TFMT5ForConditionalGeneration.

All the layers of TFMT5ForConditionalGeneration were initialized from the model checkpoint at google/mt5-small.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFMT5ForConditionalGeneration for predictions without further training.


In [31]:
loaded_model.summary()

Model: "tfmt5_for_conditional_generation_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 shared (Embedding)          multiple                  128057344 
                                                                 
 encoder (TFT5MainLayer)     multiple                  146940608 
                                                                 
 decoder (TFT5MainLayer)     multiple                  153236160 
                                                                 
 lm_head (Dense)             multiple                  128057344 
                                                                 
Total params: 300,176,768
Trainable params: 300,176,768
Non-trainable params: 0
_________________________________________________________________


In [33]:
def loaded_model_inference(txt: str):
    test_data = [txt]
    inference_tokenizer, tokenized_data = prod_inference_tokenize(input=test_data, n_tokens=INPUT_N_TOKENS)
    pred = loaded_model.generate(**tokenized_data, max_new_tokens=TARGET_N_TOKENS)
#     print(pred[0])
    result = inference_tokenizer.decode(pred[0])
    result = re.sub("<.*?>", "", result).strip()
    print(f"INPUT TEXT:\n{txt}\n\nHEADLINE:\n{result}")
    return (txt, result)

In [34]:
txt = '''
ऑलराउंडर हार्दिक पंड्या लिगामेंट में चोट लगने की वजह से वनडे वर्ल्ड कप में इंग्लैंड के खिलाफ प्लेइंग इलेवन से बाहर रह सकते हैं। यह मैच रविवार 29 अक्टूबर को लखनऊ में खेला जाएगा। वे इस मैच के लिए टीम इंडिया के साथ लखनऊ नहीं गए हैं। वे अभी नेशनल क्रिकेट एकेडमी (NCA), बेंगलुरु में हैं और अपनी चोट का इलाज करा रहे हैं।

ऐसा कहा जा रहा है कि अगर उनकी चोट ठीक नहीं होती है तो वे 2 नवंबर को मुंबई में श्रीलंका और 5 नवंबर को कोलकाता में साउथ अफ्रीका के खिलाफ होने वाले मैच से भी बाहर रह सकते हैं। हालांकि अभी तक भारतीय क्रिकेट कंट्रोल बोर्ड (BCCI) ने इसकी पुष्टि नहीं की है।

हार्दिक ने आखिरी मैच 19 अक्टूबर को पुणे में खेला था, तभी चोट लगी
हार्दिक को बांग्लादेश के खिलाफ 19 अक्टूबर को पुणे में खेले गए मैच में चोट लगी थी। 9वें ओवर की तीसरी बॉल पर उनका टखना मुड़ गया था। वह क्रीज पर ही बैठ गए थे। चोट ज्यादा होने की वजह से वे बाकी का मैच खेल नहीं पाए थे। उनका ओवर भी विराट कोहली ने पूरा किया था।
'''
txt, result = loaded_model_inference(txt)

INPUT TEXT:

ऑलराउंडर हार्दिक पंड्या लिगामेंट में चोट लगने की वजह से वनडे वर्ल्ड कप में इंग्लैंड के खिलाफ प्लेइंग इलेवन से बाहर रह सकते हैं। यह मैच रविवार 29 अक्टूबर को लखनऊ में खेला जाएगा। वे इस मैच के लिए टीम इंडिया के साथ लखनऊ नहीं गए हैं। वे अभी नेशनल क्रिकेट एकेडमी (NCA), बेंगलुरु में हैं और अपनी चोट का इलाज करा रहे हैं।

ऐसा कहा जा रहा है कि अगर उनकी चोट ठीक नहीं होती है तो वे 2 नवंबर को मुंबई में श्रीलंका और 5 नवंबर को कोलकाता में साउथ अफ्रीका के खिलाफ होने वाले मैच से भी बाहर रह सकते हैं। हालांकि अभी तक भारतीय क्रिकेट कंट्रोल बोर्ड (BCCI) ने इसकी पुष्टि नहीं की है।

हार्दिक ने आखिरी मैच 19 अक्टूबर को पुणे में खेला था, तभी चोट लगी
हार्दिक को बांग्लादेश के खिलाफ 19 अक्टूबर को पुणे में खेले गए मैच में चोट लगी थी। 9वें ओवर की तीसरी बॉल पर उनका टखना मुड़ गया था। वह क्रीज पर ही बैठ गए थे। चोट ज्यादा होने की वजह से वे बाकी का मैच खेल नहीं पाए थे। उनका ओवर भी विराट कोहली ने पूरा किया था।


HEADLINE:
मुंबई में लखनऊ में खेले गए हैं और मुंबई में बाहर रह सकते हैं भारतीय क्रिकेट एकेडमी


In [35]:
txt = '''
इलेक्शन कमीशन ने गुरुवार को केंद्र सरकार को चुनावी राज्यों में विकसित भारत संकल्प यात्रा न निकालने के निर्देश दिए। इसके लिए कमीशन ने कैबिनेट सेक्रेटरी राजीव गौबा को एक लेटर लिखा है।

कमीशन का केंद्र से कहना है कि 5 दिसंबर तक जहां आचार संहिता लागू है, वहां ये यात्रा न निकाला जाए। इस पर सरकार ने कहा कि हम इस निर्देश का पालन करेंगे।

दरअसल, राजस्थान, मध्य प्रदेश, छत्तीसगढ़, तेलंगाना और मिजोरम में विधानसभा चुनावों की घोषणा हो चुकी है। इन राज्यों में नवंबर में अलग-अलग तारीखों पर मतदान होगा और नतीजे तीन दिसंबर को घोषित किए जाएंगे।

20 नवंबर से शुरू होगी यात्रा
केंद्र सरकार ने देश भर में ‘विकसित भारत संकल्प यात्रा’ निकालने का निर्णय लिया है, जो 20 नवंबर से शुरू होगी और अगले साल 25 जनवरी तक चलेगी। यह यात्रा देश की 2.7 लाख ग्राम पंचायतों में जाएगी। इस दौरान सभी 765 जिलों में सीनियर अधिकारियों को रथ प्रभारी के तौर पर नियुक्त किया जाएगा। यात्रा में केंद्र सरकार की योजनाओं का लाभ आम जनता तक पहुंचाया जाएगा।
'''
txt, result = loaded_model_inference(txt)

INPUT TEXT:

इलेक्शन कमीशन ने गुरुवार को केंद्र सरकार को चुनावी राज्यों में विकसित भारत संकल्प यात्रा न निकालने के निर्देश दिए। इसके लिए कमीशन ने कैबिनेट सेक्रेटरी राजीव गौबा को एक लेटर लिखा है।

कमीशन का केंद्र से कहना है कि 5 दिसंबर तक जहां आचार संहिता लागू है, वहां ये यात्रा न निकाला जाए। इस पर सरकार ने कहा कि हम इस निर्देश का पालन करेंगे।

दरअसल, राजस्थान, मध्य प्रदेश, छत्तीसगढ़, तेलंगाना और मिजोरम में विधानसभा चुनावों की घोषणा हो चुकी है। इन राज्यों में नवंबर में अलग-अलग तारीखों पर मतदान होगा और नतीजे तीन दिसंबर को घोषित किए जाएंगे।

20 नवंबर से शुरू होगी यात्रा
केंद्र सरकार ने देश भर में ‘विकसित भारत संकल्प यात्रा’ निकालने का निर्णय लिया है, जो 20 नवंबर से शुरू होगी और अगले साल 25 जनवरी तक चलेगी। यह यात्रा देश की 2.7 लाख ग्राम पंचायतों में जाएगी। इस दौरान सभी 765 जिलों में सीनियर अधिकारियों को रथ प्रभारी के तौर पर नियुक्त किया जाएगा। यात्रा में केंद्र सरकार की योजनाओं का लाभ आम जनता तक पहुंचाया जाएगा।


HEADLINE:
न निकाला जाएगा नतीजे


In [36]:
txt = '''
गाजा पट्टी से इजराइल पर 7 अक्टूबर को रॉकेट हमले हुए। इसके बाद सैकड़ों हमास मेंबर्स इजराइल में घुसे और इसका नतीजा यह हुआ कि करीब 1500 इजराइली मारे गए। 224 लोगों को बंधक बनाया गया। जो अब तक हमास की कैद में हैं।

हमलों के बाद इजराइल की बेहद ताकतवर खुफिया एजेंसी मोसाद पर दुनिया तंज कस रही है तो प्रधानमंत्री बेंजामिन नेतन्याहू भी 7 अक्टूबर की नाकामी कबूल करते हैं। सवाल यह है कि हमास ने इतना बड़ा हमला कैसे किया और मोसाद को इसकी भनक क्यों नहीं लगी?

अब कुछ मीडिया रिपोर्ट्स बता रही हैं कि हमास काफी लंबे वक्त से हमले की प्लानिंग कर रहा था। उसने मोबाइल फोन की जगह लैंडलाइन फोन इस्तेमाल किए, ताकि मोसाद उसकी बातचीत न पकड़ सके।
'''
txt, result = loaded_model_inference(txt)

INPUT TEXT:

गाजा पट्टी से इजराइल पर 7 अक्टूबर को रॉकेट हमले हुए। इसके बाद सैकड़ों हमास मेंबर्स इजराइल में घुसे और इसका नतीजा यह हुआ कि करीब 1500 इजराइली मारे गए। 224 लोगों को बंधक बनाया गया। जो अब तक हमास की कैद में हैं।

हमलों के बाद इजराइल की बेहद ताकतवर खुफिया एजेंसी मोसाद पर दुनिया तंज कस रही है तो प्रधानमंत्री बेंजामिन नेतन्याहू भी 7 अक्टूबर की नाकामी कबूल करते हैं। सवाल यह है कि हमास ने इतना बड़ा हमला कैसे किया और मोसाद को इसकी भनक क्यों नहीं लगी?

अब कुछ मीडिया रिपोर्ट्स बता रही हैं कि हमास काफी लंबे वक्त से हमले की प्लानिंग कर रहा था। उसने मोबाइल फोन की जगह लैंडलाइन फोन इस्तेमाल किए, ताकि मोसाद उसकी बातचीत न पकड़ सके।


HEADLINE:
रॉकेट हमले


In [37]:
txt = '''
वन एवं पर्यावरण मंत्री हेमाराम चौधरी ने चौथी बार चुनाव लड़ने से इनकार किया है। इससे पहले वो साल 2008, 2013 ओर 2018 में चुनाव लड़ने से इनकार कर चुके हैं। गुरुवार को चौधरी ने कांग्रेस के राष्ट्रीय अध्यक्ष मल्लिकार्जुन खड़गे से अपनी जगह किसी युवा नेता को टिकट देने की मांग की है। इसके लिए उन्होंने खड़गे को लेटर भी लिखा है। वे सार्वजनिक मंच से कई बार इसकी घोषणा कर चुके हैं। गुरुवार को लेटर लिख कर उन्होंने पार्टी को अपनी मंशा बता दी है। साथ ही इसे सोशल मीडिया प्लेटफॉर्म X पर शेयर किया है।

उन्होंने लिखा है- पार्टी ने मुझे 6 बार मौका दिया। अब मैं

'''
txt, result = loaded_model_inference(txt)

INPUT TEXT:

वन एवं पर्यावरण मंत्री हेमाराम चौधरी ने चौथी बार चुनाव लड़ने से इनकार किया है। इससे पहले वो साल 2008, 2013 ओर 2018 में चुनाव लड़ने से इनकार कर चुके हैं। गुरुवार को चौधरी ने कांग्रेस के राष्ट्रीय अध्यक्ष मल्लिकार्जुन खड़गे से अपनी जगह किसी युवा नेता को टिकट देने की मांग की है। इसके लिए उन्होंने खड़गे को लेटर भी लिखा है। वे सार्वजनिक मंच से कई बार इसकी घोषणा कर चुके हैं। गुरुवार को लेटर लिख कर उन्होंने पार्टी को अपनी मंशा बता दी है। साथ ही इसे सोशल मीडिया प्लेटफॉर्म X पर शेयर किया है।

उन्होंने लिखा है- पार्टी ने मुझे 6 बार मौका दिया। अब मैं



HEADLINE:
कांग्रेस के राष्ट्रीय अध्यक्ष


In [38]:
txt = '''
BMW इंडिया ने आज (26 अक्टूबर) भारत में अपनी पॉपलुर लग्जरी कूप एसयूवी X4 का एक स्पोर्टी वैरिएंट M40i लॉन्च किया है। कंपनी का दावा है कि ये कार 4.9 सेकेंड में 0 से 100 किलोमीटर प्रति घंटा की स्पीड हासिल कर सकती है।

कंपनी ने X4 M40i को 96.20 लाख रुपए (एक्स-शोरूम) की कीमत पर पेश किया है। लक्जरी कूप एसयूवी भारत में पूरी तरह से निर्मित यूनिट (CBU) के रूप में अवेलेबल होगी। भारतीय बाजार में इसका मुकाबला मर्सिडीज-AMG GLC 43 से होगा।

कंपनी इसकी लिमिटेड यूनिट ही सेल करेगी, हालांकि बीएमडब्ल्यू ने कोई संख्या नहीं बताई है। इसे विशेष रूप से बीएमडब्ल्यू ऑनलाइन शॉप पर बुक किया जा सकता है। जर्मन ऑटो मेकर कंपनी ने कार को इस साल की शुरुआत में भारत में डिस्कंटीन्यू कर दिया था।
'''
txt, result = loaded_model_inference(txt)

INPUT TEXT:

BMW इंडिया ने आज (26 अक्टूबर) भारत में अपनी पॉपलुर लग्जरी कूप एसयूवी X4 का एक स्पोर्टी वैरिएंट M40i लॉन्च किया है। कंपनी का दावा है कि ये कार 4.9 सेकेंड में 0 से 100 किलोमीटर प्रति घंटा की स्पीड हासिल कर सकती है।

कंपनी ने X4 M40i को 96.20 लाख रुपए (एक्स-शोरूम) की कीमत पर पेश किया है। लक्जरी कूप एसयूवी भारत में पूरी तरह से निर्मित यूनिट (CBU) के रूप में अवेलेबल होगी। भारतीय बाजार में इसका मुकाबला मर्सिडीज-AMG GLC 43 से होगा।

कंपनी इसकी लिमिटेड यूनिट ही सेल करेगी, हालांकि बीएमडब्ल्यू ने कोई संख्या नहीं बताई है। इसे विशेष रूप से बीएमडब्ल्यू ऑनलाइन शॉप पर बुक किया जा सकता है। जर्मन ऑटो मेकर कंपनी ने कार को इस साल की शुरुआत में भारत में डिस्कंटीन्यू कर दिया था।


HEADLINE:
लॉन्च


In [39]:
txt = '''
इंफोसिस के को-फाउंडर एनआर नारायण मूर्ति चाहते हैं कि युवा प्रतिदिन 12 घंटे काम करें ताकि भारत उन अर्थव्यवस्थाओं के साथ प्रतिस्पर्धा कर सके जिन्होंने पिछले दो से तीन दशकों में जबरदस्त प्रोग्रेस की है। नारायण मूर्ति ने पॉडकास्ट 'द रिकॉर्ड' के लिए इंफोसिस के पूर्व CFO मोहनदास पई से बात करते हुए ये बात कही।

उन्होंने आगे कहा कि भारत की वर्क प्रोडक्टिविटी दुनिया में सबसे कम है और चीन जैसे देशों के साथ प्रतिस्पर्धा करने के लिए, देश के युवाओं को अतिरिक्त घंटे काम करना होगा जैसा कि द्वितीय विश्व युद्ध के बाद जापान और जर्मनी ने किया था। भारत की वर्क प्रोडक्टिविटी दुनिया में सबसे कम में से एक है।

पॉडकास्ट 'द रिकॉर्ड' में कहीं नारायण मूर्ति की 5 बड़ी बातें

जब तक हम अपनी वर्क प्रोडक्टिविटी में सुधार नहीं करते, जब तक हम सरकार में भ्रष्टाचार को कम नहीं करते, क्योंकि हम इसके बारे में पढ़ते रहे हैं, मुझे इसकी सच्चाई नहीं पता, जब तक हम नौकरशाही की निर्णय लेने की देरी को कम नहीं करते तब तक हम उन देशों के साथ प्रतिस्पर्धा नहीं कर सकेंगे जिन्होंने जबरदस्त प्रगति की है।
'''
txt, result = loaded_model_inference(txt)

INPUT TEXT:

इंफोसिस के को-फाउंडर एनआर नारायण मूर्ति चाहते हैं कि युवा प्रतिदिन 12 घंटे काम करें ताकि भारत उन अर्थव्यवस्थाओं के साथ प्रतिस्पर्धा कर सके जिन्होंने पिछले दो से तीन दशकों में जबरदस्त प्रोग्रेस की है। नारायण मूर्ति ने पॉडकास्ट 'द रिकॉर्ड' के लिए इंफोसिस के पूर्व CFO मोहनदास पई से बात करते हुए ये बात कही।

उन्होंने आगे कहा कि भारत की वर्क प्रोडक्टिविटी दुनिया में सबसे कम है और चीन जैसे देशों के साथ प्रतिस्पर्धा करने के लिए, देश के युवाओं को अतिरिक्त घंटे काम करना होगा जैसा कि द्वितीय विश्व युद्ध के बाद जापान और जर्मनी ने किया था। भारत की वर्क प्रोडक्टिविटी दुनिया में सबसे कम में से एक है।

पॉडकास्ट 'द रिकॉर्ड' में कहीं नारायण मूर्ति की 5 बड़ी बातें

जब तक हम अपनी वर्क प्रोडक्टिविटी में सुधार नहीं करते, जब तक हम सरकार में भ्रष्टाचार को कम नहीं करते, क्योंकि हम इसके बारे में पढ़ते रहे हैं, मुझे इसकी सच्चाई नहीं पता, जब तक हम नौकरशाही की निर्णय लेने की देरी को कम नहीं करते तब तक हम उन देशों के साथ प्रतिस्पर्धा नहीं कर सकेंगे जिन्होंने जबरदस्त प्रगति की है।


HEADLINE:
नारायण मूर